## Results for Tables 1, 2, 3 for CARLA methods

In [1]:
from carla.data.catalog import OnlineCatalog
import pandas as pd
import numpy as np

# load catalog dataset
data_name = "adult"
# data_name = "give_me_some_credit"
data_name = 'compas'
dataset = OnlineCatalog(data_name)


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/nr/samba/

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
from carla.models.catalog import MLModelCatalog
import torch
torch.manual_seed(0)

ml_model = MLModelCatalog(
        dataset, 
        model_type="ann", 
        load_online=False, 
        backend="pytorch"
    )
if data_name == 'adult':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=1024,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'give_me_some_credit':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=2048,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'compas':
    ml_model.train(
    learning_rate=0.002,
    epochs=25,
    batch_size=25,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )


balance on test set 0.8131345863037374, balance on test set 0.8191834089436163
Epoch 0/24
----------


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


train Loss: 0.5018 Acc: 0.7280

test Loss: 0.4022 Acc: 0.8192

Epoch 1/24
----------
train Loss: 0.4002 Acc: 0.8283

test Loss: 0.3704 Acc: 0.8471

Epoch 2/24
----------
train Loss: 0.3796 Acc: 0.8356

test Loss: 0.3557 Acc: 0.8419

Epoch 3/24
----------
train Loss: 0.3665 Acc: 0.8423

test Loss: 0.3405 Acc: 0.8477

Epoch 4/24
----------
train Loss: 0.3620 Acc: 0.8438

test Loss: 0.3483 Acc: 0.8464

Epoch 5/24
----------
train Loss: 0.3589 Acc: 0.8445

test Loss: 0.3409 Acc: 0.8542

Epoch 6/24
----------
train Loss: 0.3592 Acc: 0.8434

test Loss: 0.3361 Acc: 0.8555

Epoch 7/24
----------
train Loss: 0.3579 Acc: 0.8451

test Loss: 0.3358 Acc: 0.8529

Epoch 8/24
----------
train Loss: 0.3564 Acc: 0.8445

test Loss: 0.3367 Acc: 0.8548

Epoch 9/24
----------
train Loss: 0.3559 Acc: 0.8440

test Loss: 0.3383 Acc: 0.8516

Epoch 10/24
----------
train Loss: 0.3564 Acc: 0.8471

test Loss: 0.3534 Acc: 0.8542

Epoch 11/24
----------
train Loss: 0.3556 Acc: 0.8447

test Loss: 0.3376 Acc: 0.8535



In [3]:
from sklearn import metrics

if data_name == 'adult':
    y = dataset.df_test['income']
elif data_name == 'give_me_some_credit':
    y = dataset.df_test['SeriousDlqin2yrs']
elif data_name == 'compas':
    y = dataset.df_test['score']

pred = ml_model.predict_proba(dataset.df_test)
pred = [row[1] for row in pred]
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.861973140964566

In [4]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:100]

In [5]:
print((f"Factuals: {factuals.shape[0]}"))
print((f"Factuals: {(factuals.shape[0]) / dataset.df.shape[0]}"))

Factuals: 746
Factuals: 0.12086843810758263


## Compute performance metrics

In [7]:
from metrics import distance, constraint_violation, feasibility, success_rate

final_results = pd.DataFrame()
for method in ['cchvae', 'cem-vae', 'revise', 'clue', 'crud', 'face']:
    if data_name == 'adult':
        cfs = pd.read_csv("Results/adult_manifold_results.csv", index_col=0)
    elif data_name == 'give_me_some_credit':
        cfs = pd.read_csv("Results/give_me_some_credit_manifold_results.csv", index_col=0)
    elif data_name == 'compas':
        cfs = pd.read_csv("Results/compas_manifold_results.csv", index_col=0)
    factuals = predict_negative_instances(ml_model, dataset.df)
    test_factual = factuals.iloc[:100]

    df_cfs = cfs[cfs['method'] == method].drop(['method',	'data'], axis=1)

    # missing values
    nan_idx = df_cfs.index[df_cfs.isnull().any(axis=1)]
    non_nan_idx = df_cfs.index[~(df_cfs.isnull()).any(axis=1)]

    output_factuals = test_factual.copy()
    output_counterfactuals = df_cfs.copy()

    factual_without_nans = output_factuals.drop(index=nan_idx)
    counterfactuals_without_nans = output_counterfactuals.drop(index=nan_idx)

    
    # counterfactuals
    results = pd.concat([dataset.inverse_transform(counterfactuals_without_nans)])
    results['method'] = method
    results['data'] = data_name

    # distances
    distances = pd.DataFrame(distance(counterfactuals_without_nans, factual_without_nans, ml_model))
    distances.set_index(non_nan_idx, inplace=True)
    results = pd.concat([results, distances], axis=1)

    results['feasibility'] = feasibility(counterfactuals_without_nans, factual_without_nans, \
        dataset.df.columns)
        
    # violations
    violations = []

    df_decoded_cfs = dataset.inverse_transform(counterfactuals_without_nans)
    df_factuals = dataset.inverse_transform(factual_without_nans)
    
    total_violations = constraint_violation(df_decoded_cfs, df_factuals, \
        dataset.continuous, dataset.categorical, dataset.immutables)
    for x in total_violations:
        violations.append(x[0])
    results['violation'] = violations
    
    # success
    results['success'] = success_rate(counterfactuals_without_nans, ml_model, cutoff=0.5)

    # time
    results['time (seconds)'] = df_cfs['time (seconds)'].mean() 
    
    results['prediction'] = ml_model.predict_proba(counterfactuals_without_nans)[:, [1]]

    final_results = pd.concat([final_results, results], axis=0)

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change 

In [ ]:
final_results[['method', 'L0', 'L1', 'L2', 'feasibility', 'violation', 'success', 'time (seconds)']].groupby(['method']).mean()

## Here we combine CARLA and MCCE results to make Tables 1, 2, 3

In [29]:
pd.set_option('display.max_columns', None)
import pandas as pd

if data_name == 'compas':
    K = 1000
else:
    K = 10000
n_test = 100
results = pd.read_csv(f"Results/{data_name}_mcce_results_k_{K}_n_{n_test}_inverse_transform.csv", index_col=0)
results.sort_index(inplace=True)
results['data'] = data_name
results['method'] = 'mcce'
results.head(1)

,age,two_year_recid,priors_count,length_of_stay,score,L0,L1,L2,feasibility,success,violation,time (seconds),data,method,prediction,c_charge_degree,race,sex
24,27.0,1.0,4.0,2.0,0.584159,2.0,0.124013,0.011432,0.038495,1,0,74.653979,compas,mcce,0.584159,F,African-American,Male


## Load original factuals

In [30]:
pd.set_option('display.max_columns', None)
temp = pd.concat([final_results, results[final_results.columns]])

temp2 = factuals.copy()
preds = ml_model.predict_proba(temp2)
new_preds = []
for x in preds:
    new_preds.append(x[1])
temp2['prediction'] = new_preds
temp2 = dataset.inverse_transform(temp2)
temp2.head(1)
temp2['L0'] = np.nan
temp2['L1'] = np.nan
temp2['L2'] = np.nan
temp2['success'] = np.nan
temp2['violation'] = np.nan
temp2['feasibility'] = np.nan
temp2['time (seconds)'] = np.nan
temp2['method'] = 'original'
temp2['data'] = data_name

temp = pd.concat([temp, temp2.iloc[0:100][temp.columns]], axis=0)

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


In [ ]:
pd.set_option('display.max_columns', None)
# temp

In [32]:
if data_name == 'give_me_some_credit':
    cols = ['method', 'data', 'prediction', 'L0', 'L1', 'L2', 'feasibility', 'violation', 'success', 'time (seconds)'] + temp.columns[9:-1].to_list()
    temp2 = temp[cols]
elif data_name == 'adult':
    cols = ['method', 'data', 'prediction', 'L0', 'L1', 'L2', 'feasibility', 'violation', 'success', 'time (seconds)'] + temp.columns[9:16].to_list() + temp.columns[17:].to_list()
    temp2 = temp[cols]

elif data_name == 'compas':
    cols = ['method', 'data', 'prediction', 'L0', 'L1', 'L2', 'feasibility', 'violation', 'success', 'time (seconds)'] + temp.columns[0:4].to_list() + temp.columns[6:9].to_list()
    temp2 = temp[cols]
temp2.to_csv(f"Final_results/{data_name}_results_mcce_and_carla_K_{K}_n_{n_test}.csv")

In [ ]:
# temp2

## To get Adult and GMC Table 1

In [ ]:
temp = pd.read_csv("Final_results/adult_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

to_write = temp[['method', 'L0', 'L2', 'feasibility', 'violation', 'success', 'time (seconds)']].groupby(['method']).mean()

to_write.reset_index(inplace=True)

CE_N = temp.groupby(['method']).size().reset_index().rename(columns={0: 'CE_N'})
to_write = pd.concat([to_write, CE_N.CE_N], axis=1)

# to_write.sort_values(['method'], inplace=True, ascending=False)
to_write = to_write[['method', 'L0', 'L2', 'feasibility', 'violation', 'success', 'CE_N', 'time (seconds)']]

print(to_write.to_latex(index=False, float_format="%.2f", ))

In [ ]:
temp = pd.read_csv("Final_results/give_me_some_credit_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

to_write = temp[['method', 'L0', 'L2', 'feasibility', 'violation', 'success', 'time (seconds)']].groupby(['method']).mean()

to_write.reset_index(inplace=True)

CE_N = temp.groupby(['method']).size().reset_index().rename(columns={0: 'CE_N'})
to_write = pd.concat([to_write, CE_N.CE_N], axis=1)

to_write = to_write[['method', 'L0', 'L2', 'feasibility', 'violation', 'success', 'CE_N', 'time (seconds)']]

print(to_write.to_latex(index=False, float_format="%.2f", ))

In [ ]:
temp = pd.read_csv("Final_results/compas_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

to_write = temp[['method', 'L0', 'L2', 'feasibility', 'violation', 'success', 'time (seconds)']].groupby(['method']).mean()

to_write.reset_index(inplace=True)

CE_N = temp.groupby(['method']).size().reset_index().rename(columns={0: 'CE_N'})
to_write = pd.concat([to_write, CE_N.CE_N], axis=1)

# to_write.sort_values(['method'], inplace=True, ascending=False)
to_write = to_write[['method', 'L0', 'L2', 'feasibility', 'violation', 'success', 'CE_N', 'time (seconds)']]

print(to_write.to_latex(index=False, float_format="%.2f", ))

## To get Adult examples in table 2

In [ ]:
pd.set_option('display.max_columns', None)

import pandas as pd

temp = pd.read_csv("Final_results/adult_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

to_write = temp.loc[31]

to_write

In [ ]:
# print(to_write.Pred.to_latex(index=False, float_format="%.2f", ))
feature = 'marital-status'
dct = {'Married': 'M', 'Non-Married': 'NM'}
to_write[feature] = [dct[item] for item in to_write[feature]]

feature = 'native-country'
dct = {'Non-US': 'NUS', 'US': 'US'}
to_write[feature] = [dct[item] for item in to_write[feature]]

feature = 'occupation'
dct = {'Managerial-Specialist': 'MS', 'Other': 'O'}
to_write[feature] = [dct[item] for item in to_write[feature]]

feature = 'race'
dct = {'White': 'W', 'Non-White': 'NW'}
to_write[feature] = [dct[item] for item in to_write[feature]]

feature = 'relationship'
dct = {'Husband': 'H', 'Non-Husband': 'NH'}
to_write[feature] = [dct[item] for item in to_write[feature]]

feature = 'sex'
dct = {'Male': 'M'}
to_write[feature] = [dct[item] for item in to_write[feature]]


feature = 'workclass'
dct = {'Self-emp-not-inc': 'SENI', 'Private': 'P', 'Non-Private': 'NP'}
to_write[feature] = [dct[item] for item in to_write[feature]]

In [ ]:
cols = ['method', 'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', \
       'hours-per-week', 'marital-status', 'native-country', \
       'occupation', 'race', 'relationship', 'sex', 'workclass']

print(to_write[cols].to_latex(index=False, float_format="%.0f", ))

## To get GMC examples in table 3

In [ ]:
temp = pd.read_csv("Final_results/give_me_some_credit_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)
# temp.loc[263]

cols = ['method', 'prediction', 'age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']

to_write = temp[cols].loc[263]

cols = ['Method', 'Pred', 'Age', 'Unsec. Lines', 'Nb Days Past 30', 'Debt Ratio', 'Month Inc.', 'Nb Credit Lines', 'Nb Times 90 Days Late', 'Nb Real Estate Loans', 'Nb Times 60 Days Past', 'Nb Dep.']

to_write.columns = cols
# to_write.sort_values(['Method'], inplace=True, ascending=False)

# print(to_write.to_latex(index=False, float_format="%.0f", ))

print(to_write.to_latex(index=False, float_format="%.2f", ))

In [ ]:
if data_name == 'give_me_some_credit':
    features = [ 'age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']
    metric_names = ['method', 'L0', 'L1', 'violations', 'validity', 'prediction']

    temp = dataset.inverse_transform(final_results.dropna()[features])
    temp = pd.concat([final_results[metric_names], temp], axis=1)
    # temp.sort_values(temp.index.name)

In [ ]:
if data_name == 'give_me_some_credit':
    mcce_results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/give_me_some_credit_mcce_results_k_10000.csv")
    mcce_results.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    mcce_results.set_index(['index'], inplace=True)

    predictions = ml_model.predict_proba(mcce_results)
    temp3 = []
    for x in predictions:
        temp3.append(x[1]) #  >= 0.5
        
    # temp.index = final_results.index
    mcce_results['prediction'] = temp3


In [ ]:
if data_name == 'give_me_some_credit':
    mcce_results.sort_values(mcce_results.index.name, inplace=True)
    mcce_results['method'] = 'mcce'
    mcce_results.rename(columns={'success': 'validity', 'violation': 'violations'}, inplace=True)
    temp_mcce = dataset.inverse_transform(mcce_results.dropna()[features])
    temp_mcce = pd.concat([mcce_results[metric_names], temp_mcce], axis=1)
    # temp_mcce.sort_values(temp_mcce.index.name)

In [ ]:
if data_name == 'give_me_some_credit':
    pd.set_option('display.max_columns', None)
    temp2 = pd.concat([temp, temp_mcce], axis=0)

    temp2.sort_values(temp2.index.name)

    features = ['method', 'prediction', 'age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']

    temp2.loc[263][features]

In [ ]:
if data_name == 'give_me_some_credit':
    factuals = predict_negative_instances(ml_model, dataset.df)
    dataset.inverse_transform(factuals.iloc[3:4])[['age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']]

    ml_model.predict_proba(factuals.iloc[3:4])


## To get COMPAS results

In [34]:
temp = pd.read_csv("Final_results/compas_results_mcce_and_carla_K_1000_n_100.csv", index_col=0)

In [39]:

cols = ['method',  'age', 'two_year_recid', 'priors_count', 'length_of_stay', 'c_charge_degree', 'race', 'sex']

to_write = temp[cols].loc[40]

cols = ['Method', 'Age', 'Two Year Recid', 'Priors Count', 'Length of Stay', 'C Charge Degree', 'Race', 'Sex']

to_write.columns = cols
to_write.sort_values(['Method'], inplace=True, ascending=False)

print(to_write.to_latex(index=False, float_format="%.0f", ))

# print(to_write.to_latex(index=False, float_format="%.2f", ))

# round(to_write[['Method', 'Pred']], 2)

\begin{tabular}{lrrrrlll}
\toprule
   Method &  Age &  Two Year Recid &  Priors Count &  Length of Stay & C Charge Degree &              Race &   Sex \\
\midrule
   revise &   47 &               0 &             9 &             211 &               F &             Other &  Male \\
 original &   25 &               1 &             9 &               6 &               M &             Other &  Male \\
     mcce &   25 &               1 &             6 &              12 &               M &             Other &  Male \\
     face &   23 &               1 &             3 &               6 &               F &  African-American &  Male \\
     crud &  119 &               1 &             2 &             387 &               F &             Other &  Male \\
     clue &   31 &               1 &             9 &            -194 &               M &             Other &  Male \\
  cem-vae &   57 &               1 &             0 &              -1 &               F &             Other &  Male \\
   cchvae & 